The trading algorithmic model is designed to predict favorable opportunities for buying the cryptocurrency Ethereum (ETH-USD) based on binary classification using historical data spanning from January 1, 2023, to April 30, 2024. The model aims to identify instances where the average return in the next 5 days is equal to or greater than a certain percentage (X%) in this case 2.9%. The value of X% is dynamically determined based on historical data, if it is set at less than 2% it represents the 90th percentile of the series, otherwise set at the 80th percentile.

Features and Engineering

The model incorporates various engineered features to enhance the prediction accuracy. These features include moving averages, exponential moving averages, daily return, volatility, differences between moving averages and exponential moving averages, differences between open and close prices, volume-weighted average price (VWAP), relative strength index (RSI), and price momentum. Additionally, close prices of several other assets such as VIX, Tether Gold USD, Future Gold (June 24), USD/RUB, Bitcoin, S&P500, S&P400, Russell 2000, and S&P GSCI are considered to engineer similar features, with an extra feature indicating the trend of the last 15 days.

Model Training and Evaluation

The model selection process involved a grid search with XGBoost, optimizing for accuracy. Through experimentation with various thresholds, the optimal threshold was identified as 0.75. In the initial test sample (10/1/2023 – 1/30/2024), the model achieved an accuracy of 60%, while in the second test sample (1/31/2024 – 4/30/2024), it reached an accuracy of 80%. This threshold allows the user to buy Ethereum when the predicted probability is 0.75 or higher or hold/sell it when the probability is lower, based on their preferences and investment goals.

Conclusion

The trading algorithmic model utilizes advanced machine learning techniques and engineered features to predict favorable buying opportunities for Ethereum. By dynamically adjusting the threshold based on historical performance and leveraging a diverse set of features, the model aims to assist users in making informed decisions in cryptocurrency trading.


# Data gathering

In [1]:
#code to get data for ethereum, VIX,, Tether Gold USD, Future Gold (June 24) and USD/RUB daily data from dates of Jan 2023 to April 2024 from yahoo finance and store in individual dataframes

import pandas as pd
import yfinance as yf
import numpy as np

# Define the tickers and date range
tickers = ['ETH-USD', '^VIX','XAUT-USD','GC=F','USDINR=X', '^GSPC', '^MID', '^RUT','^SPGSCI', 'BTC-USD']
start_date = '2023-01-01'
end_date = '2024-04-30'

# Download the data for each ticker
data = {}
for ticker in tickers:
    data[ticker] = yf.download(ticker, start=start_date, end=end_date)

# Create individual data frames for each ticker
ETH_df = data['ETH-USD']
vix_df = data['^VIX']
xaut_df = data ['XAUT-USD']
gold_df= data['GC=F']
indusd_df = data ['USDINR=X']
bitcoin_df = data['BTC-USD']
sp500_df = data['^GSPC']
sp400_df = data['^MID']
russell2000_df = data['^RUT']
spgcci_df= data['^SPGSCI']



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [2]:
#Make date part of the table
ETH_df = ETH_df.reset_index()
vix_df = vix_df.reset_index()
xaut_df = xaut_df.reset_index()
gold_df = gold_df.reset_index()
indusd_df = indusd_df.reset_index()
bitcoin_df = bitcoin_df.reset_index()
sp500_df = sp500_df.reset_index()
sp400_df = sp400_df.reset_index()
russell2000_df = russell2000_df.reset_index()
spgcci_df = spgcci_df.reset_index()

In [3]:
ETH_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-01-01,1196.713623,1203.475342,1192.885376,1200.964844,1200.964844,2399674550
1,2023-01-02,1201.103271,1219.860596,1195.214966,1214.656616,1214.656616,3765758498
2,2023-01-03,1214.744019,1219.095337,1207.491577,1214.778809,1214.778809,3392972131
3,2023-01-04,1214.718628,1264.807495,1213.168823,1256.526611,1256.526611,6404416893
4,2023-01-05,1256.484619,1258.571533,1245.173096,1250.438599,1250.438599,4001786456


In [4]:
ETH_df.shape

(485, 7)

In [5]:
# Check for missing values
print(ETH_df.isnull().sum())

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


# Defining X and finding Target variable

In [6]:
#Calculating the average price in next 5 days
ETH_df['rolling_avg_next_5'] = ETH_df['Close'].rolling(window=5).mean().shift(-5)

# Calculating the average return in the next 5 days
ETH_df['average_return_next_5_days'] = (ETH_df['rolling_avg_next_5'] - ETH_df['Close']) / ETH_df['Close']

# Calculating moving, exponential averages, volatility in returns for 'N' days and theirs differences to add as features

In [7]:
# Calculate daily returns for ETH
ETH_df['daily_returns'] = ETH_df['Close'].pct_change()

# Moving Averages of Returns for 3, 9,12,15 and 30 days
ETH_df['MA_returns_3'] = ETH_df['daily_returns'].rolling(window=3).mean()
ETH_df['MA_returns_9'] = ETH_df['daily_returns'].rolling(window=9).mean()
ETH_df['MA_returns_12'] = ETH_df['daily_returns'].rolling(window=12).mean()
ETH_df['MA_returns_15'] = ETH_df['daily_returns'].rolling(window=15).mean()
ETH_df['MA_returns_30'] = ETH_df['daily_returns'].rolling(window=30).mean()

# Exponential Moving Averages of Returns for 3, 9, and 12 days
ETH_df['EWMA_returns_3'] = ETH_df['daily_returns'].ewm(span=3).mean()
ETH_df['EWMA_returns_9'] = ETH_df['daily_returns'].ewm(span=9).mean()
ETH_df['EWMA_returns_12'] = ETH_df['daily_returns'].ewm(span=12).mean()
ETH_df['EWMA_returns_15'] = ETH_df['daily_returns'].ewm(span=15).mean()
ETH_df['EWMA_returns_30'] = ETH_df['daily_returns'].ewm(span=30).mean()

# Volatility (Standard Deviation) of Returns for 3, 9, and 12 days
ETH_df['Volatility_returns_3'] = ETH_df['daily_returns'].rolling(window=3).std()
ETH_df['Volatility_returns_9'] = ETH_df['daily_returns'].rolling(window=9).std()
ETH_df['Volatility_returns_12'] = ETH_df['daily_returns'].rolling(window=12).std()
ETH_df['Volatility_returns_15'] = ETH_df['daily_returns'].rolling(window=15).std()
ETH_df['Volatility_returns_30'] = ETH_df['daily_returns'].rolling(window=30).std()

# Differences between MAs of Returns
ETH_df["MA_diff_9_3"] = ETH_df["MA_returns_9"] - ETH_df["MA_returns_3"]
ETH_df["MA_diff_12_9"] = ETH_df["MA_returns_12"] - ETH_df["MA_returns_9"]
ETH_df["MA_diff_12_3"] = ETH_df["MA_returns_12"] - ETH_df["MA_returns_3"]
ETH_df["MA_diff_15_12"] = ETH_df["MA_returns_15"] - ETH_df["MA_returns_12"]
ETH_df["MA_diff_15_30"] = ETH_df["MA_returns_30"] - ETH_df["MA_returns_15"]


# Differences between EWMAs of Returns
ETH_df["EWMA_diff_9_3"] = ETH_df["EWMA_returns_9"] - ETH_df["EWMA_returns_3"]
ETH_df["EWMA_diff_12_9"] = ETH_df["EWMA_returns_12"] - ETH_df["EWMA_returns_9"]
ETH_df["EWMA_diff_12_3"] = ETH_df["EWMA_returns_12"] - ETH_df["EWMA_returns_3"]
ETH_df["EWMA_diff_15_12"] = ETH_df["EWMA_returns_15"] - ETH_df["EWMA_returns_12"]
ETH_df["EWMA_diff_15_30"] = ETH_df["EWMA_returns_30"] - ETH_df["EWMA_returns_15"]




In [8]:
ETH_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,rolling_avg_next_5,average_return_next_5_days,daily_returns,...,MA_diff_9_3,MA_diff_12_9,MA_diff_12_3,MA_diff_15_12,MA_diff_15_30,EWMA_diff_9_3,EWMA_diff_12_9,EWMA_diff_12_3,EWMA_diff_15_12,EWMA_diff_15_30
0,2023-01-01,1196.713623,1203.475342,1192.885376,1200.964844,1200.964844,2399674550,1241.155933,0.033466,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-02,1201.103271,1219.860596,1195.214966,1214.656616,1214.656616,3765758498,1251.078687,0.029985,0.011401,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
2,2023-01-03,1214.744019,1219.095337,1207.491577,1214.778809,1214.778809,3392972131,1265.594824,0.041831,0.000101,...,NaN,NaN,NaN,NaN,NaN,0.001256,0.000157,0.001413,0.000094,0.000188
3,2023-01-04,1214.718628,1264.807495,1213.168823,1256.526611,1256.526611,6404416893,1278.597290,0.017565,0.034367,...,NaN,NaN,NaN,NaN,NaN,-0.004187,-0.000476,-0.004663,-0.000279,-0.000543
4,2023-01-05,1256.484619,1258.571533,1245.173096,1250.438599,1250.438599,4001786456,1295.826782,0.036298,-0.004845,...,NaN,NaN,NaN,NaN,NaN,0.002317,0.000181,0.002499,0.000097,0.000169


# Features with SQL

In [9]:
import sqlite3

conn = sqlite3.connect("my_db.db")

ETH_df.to_sql("data_in_sql", conn, if_exists="replace",index=False)

485

In [10]:
# Calculate the price difference during the day of ETH, since this is daily data we will use Open and Close
query="""
SELECT
    Date,
    Close - Open AS price_difference
FROM
    data_in_sql
ORDER BY
    Date
"""
price_difference=pd.read_sql_query(query, conn)

In [11]:
price_difference

,Date,price_difference
0,2023-01-01 00:00:00,4.251221
1,2023-01-02 00:00:00,13.553345
2,2023-01-03 00:00:00,0.034790
3,2023-01-04 00:00:00,41.807983
4,2023-01-05 00:00:00,-6.046021
...,...,...
480,2024-04-25 00:00:00,16.885254
481,2024-04-26 00:00:00,-26.219238
482,2024-04-27 00:00:00,122.441162
483,2024-04-28 00:00:00,10.529053


In [12]:
#Calculate the VWAP (Volume Weighted Average Price)
query = """
SELECT
    Date,
    SUM(Close * Volume) / SUM(Volume) AS vwap
FROM
    data_in_sql
GROUP BY
    Date;
"""
vwap = pd.read_sql_query(query, conn)

In [13]:
vwap

,Date,vwap
0,2023-01-01 00:00:00,1200.964844
1,2023-01-02 00:00:00,1214.656616
2,2023-01-03 00:00:00,1214.778809
3,2023-01-04 00:00:00,1256.526611
4,2023-01-05 00:00:00,1250.438599
...,...,...
480,2024-04-25 00:00:00,3156.509521
481,2024-04-26 00:00:00,3130.164795
482,2024-04-27 00:00:00,3252.168213
483,2024-04-28 00:00:00,3262.774658


In [14]:
#Calculate the Relative Strength Index (RSI) it helps measure the speed and change of price movements

query = """
WITH gains_losses AS (
    SELECT
        Date,
        Close - LAG(Close) OVER (ORDER BY Date) AS price_change,
        CASE WHEN Close - LAG(Close) OVER (ORDER BY Date) > 0 THEN Close - LAG(Close) OVER (ORDER BY Date) ELSE 0 END AS gain,
        CASE WHEN Close - LAG(Close) OVER (ORDER BY Date) < 0 THEN ABS(Close - LAG(Close) OVER (ORDER BY Date)) ELSE 0 END AS loss
    FROM
        data_in_sql
),
average_gains_losses AS (
    SELECT
        Date,
        AVG(gain) AS avg_gain,
        AVG(loss) AS avg_loss
    FROM
        gains_losses
    GROUP BY
        Date
)
SELECT
    Date,
    100 - (100 / (1 + (avg_gain / avg_loss))) rsi
FROM
    average_gains_losses
"""
rsi = pd.read_sql_query(query, conn)

In [15]:
rsi

,Date,rsi
0,2023-01-01 00:00:00,NaN
1,2023-01-02 00:00:00,NaN
2,2023-01-03 00:00:00,NaN
3,2023-01-04 00:00:00,NaN
4,2023-01-05 00:00:00,0.0
...,...,...
480,2024-04-25 00:00:00,NaN
481,2024-04-26 00:00:00,0.0
482,2024-04-27 00:00:00,NaN
483,2024-04-28 00:00:00,NaN


In [16]:
#calculate the Price momentum that helps deteremine the rate of change in price movement over a period of time to help investors determine the strength of a trend
query= """
WITH momentum_data AS (
    SELECT
        Date,
        close,
        LAG(close) OVER (ORDER BY Date) AS prev_close,
        close - LAG(close) OVER (ORDER BY Date) AS price_change
    FROM data_in_sql
)
SELECT
    Date,
    close,
    prev_close,
    price_change,
    price_change / prev_close AS momentum
FROM momentum_data
"""
momentum = pd.read_sql_query(query, conn)

In [17]:
momentum

,Date,close,prev_close,price_change,momentum
0,2023-01-01 00:00:00,1200.964844,NaN,NaN,NaN
1,2023-01-02 00:00:00,1214.656616,1200.964844,13.691772,0.011401
2,2023-01-03 00:00:00,1214.778809,1214.656616,0.122192,0.000101
3,2023-01-04 00:00:00,1256.526611,1214.778809,41.747803,0.034367
4,2023-01-05 00:00:00,1250.438599,1256.526611,-6.088013,-0.004845
...,...,...,...,...,...
480,2024-04-25 00:00:00,3156.509521,3139.805176,16.704346,0.005320
481,2024-04-26 00:00:00,3130.164795,3156.509521,-26.344727,-0.008346
482,2024-04-27 00:00:00,3252.168213,3130.164795,122.003418,0.038977
483,2024-04-28 00:00:00,3262.774658,3252.168213,10.606445,0.003261


# Join SQL features to the main dataframe

In [18]:
conn.execute(
    """
create table base as
SELECT Date, Open, High, Low, Close, Volume, rolling_avg_next_5,average_return_next_5_days, daily_returns, MA_returns_3, MA_returns_9, MA_returns_12, MA_returns_15, MA_returns_30, EWMA_returns_3, EWMA_returns_9, EWMA_returns_12, EWMA_returns_15, EWMA_returns_30, Volatility_returns_3, Volatility_returns_9, Volatility_returns_12, Volatility_returns_15, Volatility_returns_30, MA_diff_9_3,MA_diff_12_9, MA_diff_15_12, MA_diff_15_30, EWMA_diff_9_3, EWMA_diff_12_9, EWMA_diff_15_12, EWMA_diff_15_30
FROM data_in_sql


group by Date
    """)

In [19]:
base = pd.read_sql_query("SELECT * FROM base", conn)
base.shape

(485, 32)

In [20]:

price_difference.to_sql("price_difference", conn, index=False)

conn.execute(
      """
  create table t1 as
  SELECT a.*, b.price_difference
  FROM base as a left join price_difference as b
  on a.Date = b.Date
      """)

t1 = pd.read_sql_query("SELECT * FROM t1", conn)

print (t1.shape)

t1.tail(10)

(485, 33)


,Date,Open,High,Low,Close,Volume,rolling_avg_next_5,average_return_next_5_days,daily_returns,MA_returns_3,...,Volatility_returns_30,MA_diff_9_3,MA_diff_12_9,MA_diff_15_12,MA_diff_15_30,EWMA_diff_9_3,EWMA_diff_12_9,EWMA_diff_15_12,EWMA_diff_15_30,price_difference
475,2024-04-20 00:00:00,3059.478027,3170.672852,3021.784912,3157.627197,9918642130,3173.033398,0.004879,0.032148,0.019062,...,0.036558,-0.029710,-0.001613,0.009803,-0.000247,-0.014459,-0.002196,-0.001112,-0.000692,98.149170
476,2024-04-21 00:00:00,3157.571045,3197.506348,3119.552002,3147.288574,9394387894,3169.608643,0.007092,-0.003274,0.008891,...,0.035651,-0.011592,-0.005547,0.004861,0.002092,-0.005662,-0.001617,-0.000885,-0.000531,-10.282471
477,2024-04-22 00:00:00,3147.663574,3236.659180,3131.367432,3201.652100,12063858733,3179.711865,-0.006853,0.017273,0.015382,...,0.035811,-0.008005,-0.015102,0.003516,0.003461,-0.007741,-0.002124,-0.001294,-0.001638,53.988525
478,2024-04-23 00:00:00,3201.588623,3264.419434,3154.589844,3219.911621,11054442653,3188.284473,-0.009822,0.005703,0.006567,...,0.035183,-0.004178,-0.008734,-0.002152,0.006759,-0.004327,-0.001867,-0.001237,-0.001831,18.322998
479,2024-04-24 00:00:00,3219.957031,3292.921631,3105.982178,3139.805176,14000234760,3203.409229,0.020257,-0.024878,-0.000634,...,0.034541,0.002207,-0.003757,-0.004541,0.002844,0.006645,-0.000225,-0.000289,-0.000125,-80.151855
480,2024-04-25 00:00:00,3139.624268,3190.976318,3074.804688,3156.509521,13989030260,NaN,NaN,0.005320,-0.004618,...,0.034578,0.007380,0.001617,-0.011482,0.003431,0.001310,-0.000500,-0.000453,-0.000554,16.885254
481,2024-04-26 00:00:00,3156.384033,3166.188721,3103.104004,3130.164795,10622333862,NaN,NaN,-0.008346,-0.009301,...,0.034371,0.014744,-0.005976,-0.006402,0.003797,0.003145,-0.000040,-0.000171,-0.000074,-26.219238
482,2024-04-27 00:00:00,3129.727051,3279.451172,3071.340088,3252.168213,11820785577,NaN,NaN,0.038977,0.011984,...,0.035033,-0.005237,-0.002571,-0.003526,-0.003072,-0.010632,-0.001911,-0.001325,-0.002542,122.441162
483,2024-04-28 00:00:00,3252.245605,3351.176514,3249.149170,3262.774658,11379192678,NaN,NaN,0.003261,0.011297,...,0.034980,-0.003944,-0.002458,0.000867,-0.007612,-0.004365,-0.001471,-0.001123,-0.002382,10.529053
484,2024-04-29 00:00:00,3262.340820,3285.468750,3116.199951,3215.428955,15032246816,NaN,NaN,-0.014511,0.009242,...,0.035055,-0.007073,0.004223,-0.004971,-0.003719,0.003910,-0.000307,-0.000439,-0.001157,-46.911865


In [21]:
t1[t1.price_difference.notnull()].shape[0]

485

In [22]:
vwap.to_sql("vwap", conn, index=False)

conn.execute(
    """
create table t2 as
SELECT a.*, b.vwap
FROM t1 as a left join vwap as b
on a.Date = b.Date
    """)

t2 = pd.read_sql_query("SELECT * FROM t2", conn)

print (t2.shape)

t2.tail(10)

(485, 34)


,Date,Open,High,Low,Close,Volume,rolling_avg_next_5,average_return_next_5_days,daily_returns,MA_returns_3,...,MA_diff_9_3,MA_diff_12_9,MA_diff_15_12,MA_diff_15_30,EWMA_diff_9_3,EWMA_diff_12_9,EWMA_diff_15_12,EWMA_diff_15_30,price_difference,vwap
475,2024-04-20 00:00:00,3059.478027,3170.672852,3021.784912,3157.627197,9918642130,3173.033398,0.004879,0.032148,0.019062,...,-0.029710,-0.001613,0.009803,-0.000247,-0.014459,-0.002196,-0.001112,-0.000692,98.149170,3157.627197
476,2024-04-21 00:00:00,3157.571045,3197.506348,3119.552002,3147.288574,9394387894,3169.608643,0.007092,-0.003274,0.008891,...,-0.011592,-0.005547,0.004861,0.002092,-0.005662,-0.001617,-0.000885,-0.000531,-10.282471,3147.288574
477,2024-04-22 00:00:00,3147.663574,3236.659180,3131.367432,3201.652100,12063858733,3179.711865,-0.006853,0.017273,0.015382,...,-0.008005,-0.015102,0.003516,0.003461,-0.007741,-0.002124,-0.001294,-0.001638,53.988525,3201.652100
478,2024-04-23 00:00:00,3201.588623,3264.419434,3154.589844,3219.911621,11054442653,3188.284473,-0.009822,0.005703,0.006567,...,-0.004178,-0.008734,-0.002152,0.006759,-0.004327,-0.001867,-0.001237,-0.001831,18.322998,3219.911621
479,2024-04-24 00:00:00,3219.957031,3292.921631,3105.982178,3139.805176,14000234760,3203.409229,0.020257,-0.024878,-0.000634,...,0.002207,-0.003757,-0.004541,0.002844,0.006645,-0.000225,-0.000289,-0.000125,-80.151855,3139.805176
480,2024-04-25 00:00:00,3139.624268,3190.976318,3074.804688,3156.509521,13989030260,NaN,NaN,0.005320,-0.004618,...,0.007380,0.001617,-0.011482,0.003431,0.001310,-0.000500,-0.000453,-0.000554,16.885254,3156.509521
481,2024-04-26 00:00:00,3156.384033,3166.188721,3103.104004,3130.164795,10622333862,NaN,NaN,-0.008346,-0.009301,...,0.014744,-0.005976,-0.006402,0.003797,0.003145,-0.000040,-0.000171,-0.000074,-26.219238,3130.164795
482,2024-04-27 00:00:00,3129.727051,3279.451172,3071.340088,3252.168213,11820785577,NaN,NaN,0.038977,0.011984,...,-0.005237,-0.002571,-0.003526,-0.003072,-0.010632,-0.001911,-0.001325,-0.002542,122.441162,3252.168213
483,2024-04-28 00:00:00,3252.245605,3351.176514,3249.149170,3262.774658,11379192678,NaN,NaN,0.003261,0.011297,...,-0.003944,-0.002458,0.000867,-0.007612,-0.004365,-0.001471,-0.001123,-0.002382,10.529053,3262.774658
484,2024-04-29 00:00:00,3262.340820,3285.468750,3116.199951,3215.428955,15032246816,NaN,NaN,-0.014511,0.009242,...,-0.007073,0.004223,-0.004971,-0.003719,0.003910,-0.000307,-0.000439,-0.001157,-46.911865,3215.428955


In [23]:
t2[t2.vwap.notnull()].shape[0]

485

In [24]:
rsi.to_sql("rsi", conn, index=False)

conn.execute(
    """
create table t3 as
SELECT a.*, b.rsi
FROM t2 as a left join rsi as b
on a.Date = b.Date
    """)

t3 = pd.read_sql_query("SELECT * FROM t3", conn)

print (t3.shape)

t3.tail(10)

(485, 35)


,Date,Open,High,Low,Close,Volume,rolling_avg_next_5,average_return_next_5_days,daily_returns,MA_returns_3,...,MA_diff_12_9,MA_diff_15_12,MA_diff_15_30,EWMA_diff_9_3,EWMA_diff_12_9,EWMA_diff_15_12,EWMA_diff_15_30,price_difference,vwap,rsi
475,2024-04-20 00:00:00,3059.478027,3170.672852,3021.784912,3157.627197,9918642130,3173.033398,0.004879,0.032148,0.019062,...,-0.001613,0.009803,-0.000247,-0.014459,-0.002196,-0.001112,-0.000692,98.149170,3157.627197,NaN
476,2024-04-21 00:00:00,3157.571045,3197.506348,3119.552002,3147.288574,9394387894,3169.608643,0.007092,-0.003274,0.008891,...,-0.005547,0.004861,0.002092,-0.005662,-0.001617,-0.000885,-0.000531,-10.282471,3147.288574,0.0
477,2024-04-22 00:00:00,3147.663574,3236.659180,3131.367432,3201.652100,12063858733,3179.711865,-0.006853,0.017273,0.015382,...,-0.015102,0.003516,0.003461,-0.007741,-0.002124,-0.001294,-0.001638,53.988525,3201.652100,NaN
478,2024-04-23 00:00:00,3201.588623,3264.419434,3154.589844,3219.911621,11054442653,3188.284473,-0.009822,0.005703,0.006567,...,-0.008734,-0.002152,0.006759,-0.004327,-0.001867,-0.001237,-0.001831,18.322998,3219.911621,NaN
479,2024-04-24 00:00:00,3219.957031,3292.921631,3105.982178,3139.805176,14000234760,3203.409229,0.020257,-0.024878,-0.000634,...,-0.003757,-0.004541,0.002844,0.006645,-0.000225,-0.000289,-0.000125,-80.151855,3139.805176,0.0
480,2024-04-25 00:00:00,3139.624268,3190.976318,3074.804688,3156.509521,13989030260,NaN,NaN,0.005320,-0.004618,...,0.001617,-0.011482,0.003431,0.001310,-0.000500,-0.000453,-0.000554,16.885254,3156.509521,NaN
481,2024-04-26 00:00:00,3156.384033,3166.188721,3103.104004,3130.164795,10622333862,NaN,NaN,-0.008346,-0.009301,...,-0.005976,-0.006402,0.003797,0.003145,-0.000040,-0.000171,-0.000074,-26.219238,3130.164795,0.0
482,2024-04-27 00:00:00,3129.727051,3279.451172,3071.340088,3252.168213,11820785577,NaN,NaN,0.038977,0.011984,...,-0.002571,-0.003526,-0.003072,-0.010632,-0.001911,-0.001325,-0.002542,122.441162,3252.168213,NaN
483,2024-04-28 00:00:00,3252.245605,3351.176514,3249.149170,3262.774658,11379192678,NaN,NaN,0.003261,0.011297,...,-0.002458,0.000867,-0.007612,-0.004365,-0.001471,-0.001123,-0.002382,10.529053,3262.774658,NaN
484,2024-04-29 00:00:00,3262.340820,3285.468750,3116.199951,3215.428955,15032246816,NaN,NaN,-0.014511,0.009242,...,0.004223,-0.004971,-0.003719,0.003910,-0.000307,-0.000439,-0.001157,-46.911865,3215.428955,0.0


In [25]:
momentum.to_sql("momentum", conn, index=False)

conn.execute(
    """
create table t4 as
SELECT a.*, b.momentum
FROM t3 as a left join momentum as b
on a.Date = b.Date
    """)

t4 = pd.read_sql_query("SELECT * FROM t4", conn)

print (t4.shape)

t4.tail(10)

(485, 36)


,Date,Open,High,Low,Close,Volume,rolling_avg_next_5,average_return_next_5_days,daily_returns,MA_returns_3,...,MA_diff_15_12,MA_diff_15_30,EWMA_diff_9_3,EWMA_diff_12_9,EWMA_diff_15_12,EWMA_diff_15_30,price_difference,vwap,rsi,momentum
475,2024-04-20 00:00:00,3059.478027,3170.672852,3021.784912,3157.627197,9918642130,3173.033398,0.004879,0.032148,0.019062,...,0.009803,-0.000247,-0.014459,-0.002196,-0.001112,-0.000692,98.149170,3157.627197,NaN,0.032148
476,2024-04-21 00:00:00,3157.571045,3197.506348,3119.552002,3147.288574,9394387894,3169.608643,0.007092,-0.003274,0.008891,...,0.004861,0.002092,-0.005662,-0.001617,-0.000885,-0.000531,-10.282471,3147.288574,0.0,-0.003274
477,2024-04-22 00:00:00,3147.663574,3236.659180,3131.367432,3201.652100,12063858733,3179.711865,-0.006853,0.017273,0.015382,...,0.003516,0.003461,-0.007741,-0.002124,-0.001294,-0.001638,53.988525,3201.652100,NaN,0.017273
478,2024-04-23 00:00:00,3201.588623,3264.419434,3154.589844,3219.911621,11054442653,3188.284473,-0.009822,0.005703,0.006567,...,-0.002152,0.006759,-0.004327,-0.001867,-0.001237,-0.001831,18.322998,3219.911621,NaN,0.005703
479,2024-04-24 00:00:00,3219.957031,3292.921631,3105.982178,3139.805176,14000234760,3203.409229,0.020257,-0.024878,-0.000634,...,-0.004541,0.002844,0.006645,-0.000225,-0.000289,-0.000125,-80.151855,3139.805176,0.0,-0.024878
480,2024-04-25 00:00:00,3139.624268,3190.976318,3074.804688,3156.509521,13989030260,NaN,NaN,0.005320,-0.004618,...,-0.011482,0.003431,0.001310,-0.000500,-0.000453,-0.000554,16.885254,3156.509521,NaN,0.005320
481,2024-04-26 00:00:00,3156.384033,3166.188721,3103.104004,3130.164795,10622333862,NaN,NaN,-0.008346,-0.009301,...,-0.006402,0.003797,0.003145,-0.000040,-0.000171,-0.000074,-26.219238,3130.164795,0.0,-0.008346
482,2024-04-27 00:00:00,3129.727051,3279.451172,3071.340088,3252.168213,11820785577,NaN,NaN,0.038977,0.011984,...,-0.003526,-0.003072,-0.010632,-0.001911,-0.001325,-0.002542,122.441162,3252.168213,NaN,0.038977
483,2024-04-28 00:00:00,3252.245605,3351.176514,3249.149170,3262.774658,11379192678,NaN,NaN,0.003261,0.011297,...,0.000867,-0.007612,-0.004365,-0.001471,-0.001123,-0.002382,10.529053,3262.774658,NaN,0.003261
484,2024-04-29 00:00:00,3262.340820,3285.468750,3116.199951,3215.428955,15032246816,NaN,NaN,-0.014511,0.009242,...,-0.004971,-0.003719,0.003910,-0.000307,-0.000439,-0.001157,-46.911865,3215.428955,0.0,-0.014511


In [26]:
t4[t4.momentum.notnull()].shape[0]

484

In [27]:
ETH_df = t4.copy()

# Now making a function that will calculate features for all the other assets.

In [28]:
from scipy.stats import linregress
def calculate_features(df):
    # Basic calculations
    df['daily_returns'] = df['Close'].pct_change()

    # Moving Averages, Exponential Moving Averages, and Volatility
    for window in [3, 9, 12, 15, 30]:
        df[f'MA_returns_{window}'] = df['daily_returns'].rolling(window=window).mean()
        df[f'Volatility_returns_{window}'] = df['daily_returns'].rolling(window=window).std()
        df[f'EWMA_returns_{window}'] = df['daily_returns'].ewm(span=window).mean()

    # Differences in MAs and EWMAs
    for window1, window2 in [(9, 3), (12, 9), (12, 3), (15, 12), (15, 30)]:
        df[f"MA_diff_{window1}_{window2}"] = df[f"MA_returns_{window1}"] - df[f"MA_returns_{window2}"]
        df[f"EWMA_diff_{window1}_{window2}"] = df[f"EWMA_returns_{window1}"] - df[f"EWMA_returns_{window2}"]

    # Trend Features
    df['trend_15_days'] = df['daily_returns'].rolling(window=15).apply(lambda x: linregress(np.arange(len(x)), x).slope, raw=False)

    return df





# Using the created function to calculate the new features for all the assets

In [29]:
sp500_df_features= calculate_features(sp500_df)
sp400_df_features= calculate_features(sp400_df)
russell2000_df_features= calculate_features(russell2000_df)
vix_df_features= calculate_features(vix_df)
spgcci_df_features= calculate_features(spgcci_df)
xaut_df_features = calculate_features (xaut_df)
gold_df_features= calculate_features (gold_df)
indusd_df_features = calculate_features(indusd_df)
bitcoin_df_features = calculate_features(bitcoin_df)


## Renaming all of the columns in each Dataframe to make columns specific before merging into the bitcoin dataset.


In [30]:

words_toAdd = ['sp500_df', 'sp400_df','russell2000','vix_df','spgcci_df', 'xaut_df', 'gold_df','indusd_df','bitcoin_df']

# Loop through each DataFrame along with the words to add
for i, (df, word) in enumerate(zip([sp500_df_features,sp400_df_features,russell2000_df_features,vix_df_features,spgcci_df_features, xaut_df_features, gold_df_features, indusd_df_features, bitcoin_df_features], words_toAdd)):
    # Loop through each column and modify column name
    for column in df.columns:
        df.rename(columns={column: word + '_' + column}, inplace=True)

    # Print the modified DataFrame
    #print(f"DataFrame {i+1} with prefix '{word}':\n{df}\n")




In [31]:
final_assets_df= ETH_df.join(sp500_df_features, how='inner')

In [32]:
final_assets_df= final_assets_df.join(sp400_df_features, how='inner')
final_assets_df= final_assets_df.join(russell2000_df_features, how='inner')
final_assets_df= final_assets_df.join(vix_df_features, how='inner')
final_assets_df= final_assets_df.join(spgcci_df_features, how='inner')
final_assets_df= final_assets_df.join(xaut_df_features, how='inner')
final_assets_df= final_assets_df.join(gold_df_features, how='inner')
final_assets_df= final_assets_df.join(indusd_df_features, how='inner')
final_assets_df= final_assets_df.join(bitcoin_df_features, how='inner')

# Fixing the threshold for target variable

In [33]:
# Define X as 80th percentile or 90th percentile
X = final_assets_df['average_return_next_5_days'].quantile(0.8)
if X < 0.02:
    X = final_assets_df['average_return_next_5_days'].quantile(0.9)

In [34]:
X

0.02913227845164388

# Define Target

In [35]:
# Define target variable
final_assets_df['target'] = (final_assets_df['average_return_next_5_days'] >= X).shift(-5).dropna()

<ipython-input-35-27b37f968f41>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_assets_df['target'] = (final_assets_df['average_return_next_5_days'] >= X).shift(-5).dropna()


In [36]:
final_assets_df.head(10)

,Date,Open,High,Low,Close,Volume,rolling_avg_next_5,average_return_next_5_days,daily_returns,MA_returns_3,...,bitcoin_df_MA_diff_12_9,bitcoin_df_EWMA_diff_12_9,bitcoin_df_MA_diff_12_3,bitcoin_df_EWMA_diff_12_3,bitcoin_df_MA_diff_15_12,bitcoin_df_EWMA_diff_15_12,bitcoin_df_MA_diff_15_30,bitcoin_df_EWMA_diff_15_30,bitcoin_df_trend_15_days,target
0,2023-01-01 00:00:00,1196.713623,1203.475342,1192.885376,1200.964844,2399674550,1241.155933,0.033466,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,2023-01-02 00:00:00,1201.103271,1219.860596,1195.214966,1214.656616,3765758498,1251.078687,0.029985,0.011401,NaN,...,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,True
2,2023-01-03 00:00:00,1214.744019,1219.095337,1207.491577,1214.778809,3392972131,1265.594824,0.041831,0.000101,NaN,...,NaN,0.000060,NaN,0.000541,NaN,0.000036,NaN,-0.000072,NaN,True
3,2023-01-04 00:00:00,1214.718628,1264.807495,1213.168823,1256.526611,6404416893,1278.597290,0.017565,0.034367,0.015289,...,NaN,-0.000151,NaN,-0.001494,NaN,-0.000088,NaN,0.000171,NaN,True
4,2023-01-05 00:00:00,1256.484619,1258.571533,1245.173096,1250.438599,4001786456,1295.826782,0.036298,-0.004845,0.009874,...,NaN,0.000056,NaN,0.000774,NaN,0.000030,NaN,-0.000053,NaN,True
5,2023-01-06 00:00:00,1250.458984,1273.220337,1240.947510,1269.379028,4977252792,1319.537524,0.039514,0.015147,0.014890,...,NaN,-0.000058,NaN,-0.000552,NaN,-0.000035,NaN,0.000068,NaN,True
6,2023-01-07 00:00:00,1269.419800,1270.598267,1262.271118,1264.270386,2565213548,1350.271143,0.068024,-0.004025,0.002092,...,NaN,0.000067,NaN,0.000761,NaN,0.000037,NaN,-0.000065,NaN,True
7,2023-01-08 00:00:00,1264.181885,1287.359497,1260.226807,1287.359497,3495088905,1383.122217,0.074387,0.018263,0.009795,...,NaN,-0.000096,NaN,-0.000995,NaN,-0.000055,NaN,0.000105,NaN,True
8,2023-01-09 00:00:00,1287.450806,1342.144165,1286.680786,1321.538940,7990438824,1428.955811,0.081282,0.026550,0.013596,...,NaN,-0.000135,NaN,-0.001065,NaN,-0.000080,NaN,0.000158,NaN,False
9,2023-01-10 00:00:00,1321.395508,1342.757202,1318.527222,1336.586060,5830173253,1472.134497,0.101414,0.011386,0.018733,...,NaN,-0.000432,NaN,-0.003530,NaN,-0.000255,NaN,0.000492,NaN,False


In [37]:
ETH_df = final_assets_df.dropna(subset=['target'])

# Now, ETH_df does not contain rows where 'target' is NaN.


In [38]:
ETH_df['target']=ETH_df['target'].astype(int) #converting Target variable to zeroes and ones from boolean values.

<ipython-input-38-58a43c63471a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ETH_df['target']=ETH_df['target'].astype(int) #converting Target variable to zeroes and ones from boolean values.


In [39]:
ETH_df.shape

(327, 343)

# Data splitting

In [40]:
# Define split dates
split_date1 = '2023-10-1 00:00:00'  # First split date
split_date2 = '2024-01-31 00:00:00'  # Second split date

# Split the data into train, and tests sets
Train = ETH_df[ETH_df.sp500_df_Date < split_date1]
Test1 = ETH_df[(ETH_df.sp500_df_Date >= split_date1) & (ETH_df.sp500_df_Date < split_date2)]
Test2 = ETH_df[ETH_df.sp500_df_Date >= split_date2]


In [41]:
Y_train = Train.target
X_train = Train.drop(["Date", "target",'sp500_df_Date', 'sp400_df_Date', 'russell2000_Date', 'vix_df_Date', 'spgcci_df_Date', 'xaut_df_Date', 'gold_df_Date', 'indusd_df_Date', 'bitcoin_df_Date'], axis = 1)

Y_test_1 = Test1.target
X_test_1 = Test1.drop(["Date", "target", 'sp500_df_Date', 'sp400_df_Date', 'russell2000_Date', 'vix_df_Date', 'spgcci_df_Date', 'xaut_df_Date', 'gold_df_Date', 'indusd_df_Date', 'bitcoin_df_Date'], axis = 1)

Y_test_2 = Test2.target
X_test_2 = Test2.drop(["Date", "target", 'sp500_df_Date', 'sp400_df_Date', 'russell2000_Date', 'vix_df_Date', 'spgcci_df_Date', 'xaut_df_Date', 'gold_df_Date', 'indusd_df_Date', 'bitcoin_df_Date'], axis = 1)

In [42]:
Y_train.shape[0]

187

In [43]:
X_test_1.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'rolling_avg_next_5',
       'average_return_next_5_days', 'daily_returns', 'MA_returns_3',
       'MA_returns_9',
       ...
       'bitcoin_df_EWMA_diff_9_3', 'bitcoin_df_MA_diff_12_9',
       'bitcoin_df_EWMA_diff_12_9', 'bitcoin_df_MA_diff_12_3',
       'bitcoin_df_EWMA_diff_12_3', 'bitcoin_df_MA_diff_15_12',
       'bitcoin_df_EWMA_diff_15_12', 'bitcoin_df_MA_diff_15_30',
       'bitcoin_df_EWMA_diff_15_30', 'bitcoin_df_trend_15_days'],
      dtype='object', length=332)

In [44]:
print (X_train.shape)
print (X_test_1.shape)
print (X_test_2.shape)

(187, 332)
(83, 332)
(57, 332)


In [45]:
Y_test_1

187    1
188    0
189    0
190    0
191    0
      ..
265    0
266    0
267    0
268    0
269    0
Name: target, Length: 83, dtype: int64

# We will use SHAP analysis instead of feature importance to get the most relevant features.



In [46]:
import xgboost as xgb
xgb_instance = xgb.XGBClassifier(n_estimators = 20)
model_feature_importance = xgb_instance.fit(X_train, Y_train)

In [47]:
# prompt: install shap

!pip install shap


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 2.9 MB/s eta 0:00:00


In [48]:
# code that will show SHAP analysis

import shap
explainer = shap.TreeExplainer(xgb_instance)
shap_values = explainer.shap_values(X_train)
shap_summary = pd.DataFrame(shap_values, columns=X_train.columns)

# Calculate the mean absolute value of the SHAP values for each feature
shap_importance = shap_summary.apply(np.abs).mean().sort_values(ascending=False)

# Print the feature importance
print(shap_importance)


bitcoin_df_MA_returns_15       0.485146
bitcoin_df_MA_diff_15_30       0.452686
spgcci_df_MA_diff_9_3          0.425288
spgcci_df_EWMA_diff_12_9       0.373473
gold_df_trend_15_days          0.311892
                                 ...   
russell2000_MA_diff_12_9       0.000000
russell2000_EWMA_diff_9_3      0.000000
russell2000_MA_diff_9_3        0.000000
russell2000_EWMA_returns_30    0.000000
bitcoin_df_trend_15_days       0.000000
Length: 332, dtype: float32


In [49]:
shap_importance

bitcoin_df_MA_returns_15       0.485146
bitcoin_df_MA_diff_15_30       0.452686
spgcci_df_MA_diff_9_3          0.425288
spgcci_df_EWMA_diff_12_9       0.373473
gold_df_trend_15_days          0.311892
                                 ...   
russell2000_MA_diff_12_9       0.000000
russell2000_EWMA_diff_9_3      0.000000
russell2000_MA_diff_9_3        0.000000
russell2000_EWMA_returns_30    0.000000
bitcoin_df_trend_15_days       0.000000
Length: 332, dtype: float32

In [50]:
# Select the top N features
N = 20
top_features = shap_importance.head(N).index.tolist()


# Subset your training and test sets to include only the top features
X_train = X_train[top_features]
X_test_1= X_test_1[top_features]
X_test_2= X_test_2[top_features]


In [52]:
top_features

['bitcoin_df_MA_returns_15',
 'bitcoin_df_MA_diff_15_30',
 'spgcci_df_MA_diff_9_3',
 'spgcci_df_EWMA_diff_12_9',
 'gold_df_trend_15_days',
 'MA_diff_15_30',
 'sp500_df_Volatility_returns_3',
 'vix_df_MA_diff_15_12',
 'bitcoin_df_MA_returns_3',
 'spgcci_df_EWMA_returns_15',
 'indusd_df_Volatility_returns_3',
 'russell2000_Volatility_returns_30',
 'Volume',
 'xaut_df_MA_returns_30',
 'russell2000_MA_returns_30',
 'spgcci_df_MA_diff_12_3',
 'spgcci_df_MA_returns_30',
 'russell2000_High',
 'sp500_df_MA_returns_3',
 'Volatility_returns_12']

In [53]:
print (X_train.shape)
print (X_test_1.shape)
print (X_test_2.shape)

(187, 20)
(83, 20)
(57, 20)


In [54]:
X_train

,bitcoin_df_MA_returns_15,bitcoin_df_MA_diff_15_30,spgcci_df_MA_diff_9_3,spgcci_df_EWMA_diff_12_9,gold_df_trend_15_days,MA_diff_15_30,sp500_df_Volatility_returns_3,vix_df_MA_diff_15_12,bitcoin_df_MA_returns_3,spgcci_df_EWMA_returns_15,indusd_df_Volatility_returns_3,russell2000_Volatility_returns_30,Volume,xaut_df_MA_returns_30,russell2000_MA_returns_30,spgcci_df_MA_diff_12_3,spgcci_df_MA_returns_30,russell2000_High,sp500_df_MA_returns_3,Volatility_returns_12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2399674550,NaN,NaN,NaN,NaN,1785.949951,NaN,NaN
1,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,-0.032829,NaN,NaN,3765758498,NaN,NaN,NaN,NaN,1781.969971,NaN,NaN
2,NaN,NaN,NaN,-0.000453,NaN,NaN,NaN,NaN,NaN,-0.015423,NaN,NaN,3392972131,NaN,NaN,NaN,NaN,1766.079956,NaN,NaN
3,NaN,NaN,NaN,-0.000637,NaN,NaN,0.017280,NaN,0.004764,-0.008150,0.001213,NaN,6404416893,NaN,NaN,NaN,NaN,1795.560059,0.006245,NaN
4,NaN,NaN,NaN,-0.000881,NaN,NaN,0.017630,NaN,0.002969,-0.002026,0.001202,NaN,4001786456,NaN,NaN,NaN,NaN,1817.089966,0.003476,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,0.009885,0.005697,0.002298,0.000660,0.000156,-0.007101,0.010456,0.003529,0.001918,0.000072,0.000806,0.009023,6343966490,-0.000551,-0.002490,0.003612,0.001044,1788.030029,-0.004891,0.025033
183,0.011490,0.006506,0.000953,0.000521,-0.000213,-0.007406,0.009544,-0.006409,0.007400,0.000034,0.001311,0.009213,7858509087,-0.000444,-0.002833,0.001954,0.001285,1786.050049,-0.004336,0.019962
184,0.009377,0.004851,-0.004091,-0.000301,-0.000665,-0.006391,0.009918,-0.008110,0.002112,0.001928,0.001344,0.009290,5683423776,-0.000426,-0.002077,-0.002625,0.002296,1786.599976,-0.003494,0.019964
185,0.005140,-0.000770,-0.003998,0.000046,-0.000737,-0.002466,0.010658,-0.004477,-0.001075,0.001166,0.000246,0.009263,6034088075,-0.000712,-0.001362,-0.003127,0.002382,1802.609985,-0.002871,0.020331


# Run XGBoost grid search


In [55]:
from sklearn.metrics import roc_auc_score
import pandas as pd
import xgboost as xgb

# Initialize a DataFrame to store the results
table = pd.DataFrame(columns=["Num Trees", "Learning Rate","max_depth","Weight","AUC Train", "AUC Test1", "AUC Test2"])

row = 0
for num_trees in [20,50,100,200]:
    for lr in [0.001, 0.01, 0.1]:
       for max_depth in [3, 4, 5]:
              for weight in [1, 5, 10]:
                # Initialize XGBoost classifier with specified parameters
                xgb_instance = xgb.XGBClassifier(n_estimators=num_trees,
                                                 learning_rate=lr,
                                                 max_depth= max_depth,
                                                 scale_pos_weight=weight)
                # Fit the model on training data
                model = xgb_instance.fit(X_train, Y_train)

                # Calculate AUC scores for training and testing sets
                auc_train = roc_auc_score(Y_train, model.predict_proba(X_train)[:, 1])
                auc_test1 = roc_auc_score(Y_test_1, model.predict_proba(X_test_1)[:, 1])
                auc_test2 = roc_auc_score(Y_test_2, model.predict_proba(X_test_2)[:, 1])

                # Store the results in the DataFrame
                table.loc[row, "Num Trees"] = num_trees
                table.loc[row, "Learning Rate"] = lr
                table.loc[row, "max_depth"] = max_depth
                table.loc[row, "Weight"] = weight
                table.loc[row, "AUC Train"] = auc_train
                table.loc[row, "AUC Test1"] = auc_test1
                table.loc[row, "AUC Test2"] = auc_test2

                row += 1

In [56]:
table

,Num Trees,Learning Rate,max_depth,Weight,AUC Train,AUC Test1,AUC Test2
0,20,0.001,3,1,0.886833,0.391026,0.519444
1,20,0.001,3,5,0.9142,0.420513,0.580556
2,20,0.001,3,10,0.85831,0.520513,0.623148
3,20,0.001,4,1,0.94095,0.384615,0.549074
4,20,0.001,4,5,0.954209,0.423077,0.677778
...,...,...,...,...,...,...,...
103,200,0.1,4,5,1.0,0.333333,0.625926
104,200,0.1,4,10,1.0,0.364103,0.703704
105,200,0.1,5,1,1.0,0.328205,0.603704
106,200,0.1,5,5,1.0,0.366667,0.646296


# Test the chosen model (number 62) with different thresholds


In [57]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score

xgb_instance = xgb.XGBClassifier(n_estimators = 100, max_depth=5, learning_rate=0.001, scale_pos_weight=10)
model_final = xgb_instance.fit(X_train, Y_train)

In [58]:
model_final_predict_proba_np = model_final.predict_proba(X_train)

In [59]:
model_final_predict_proba_np_default = model_final_predict_proba_np[:,1]

In [60]:
Train["Prediction_Prob"] = model_final_predict_proba_np_default

<ipython-input-60-1cfc1965c7bc>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Train["Prediction_Prob"] = model_final_predict_proba_np_default
<ipython-input-60-1cfc1965c7bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Train["Prediction_Prob"] = model_final_predict_proba_np_default


In [61]:
Train[["target", "Prediction_Prob",]].head(10)

,target,Prediction_Prob
0,1,0.764260
1,1,0.764260
2,1,0.764260
3,1,0.764260
4,1,0.764260
5,1,0.764260
6,1,0.764260
7,1,0.764260
8,0,0.693435
9,0,0.693435


In [62]:
model_final_predict_proba_np_t1 = model_final.predict_proba(X_test_1)
model_final_predict_proba_np_default_t1 = model_final_predict_proba_np_t1[:,1]

In [63]:
Test1["Prediction_Prob"] = model_final_predict_proba_np_default_t1

<ipython-input-63-a3893b9ab93f>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Test1["Prediction_Prob"] = model_final_predict_proba_np_default_t1
<ipython-input-63-a3893b9ab93f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test1["Prediction_Prob"] = model_final_predict_proba_np_default_t1


In [64]:
Test1[["target", "Prediction_Prob",]].tail(10)

,target,Prediction_Prob
260,0,0.693435
261,0,0.693435
262,1,0.693435
263,1,0.688685
264,1,0.764260
265,0,0.764260
266,0,0.746368
267,0,0.764260
268,0,0.684759
269,0,0.764260


In [65]:
model_final_predict_proba_np_t2 = model_final.predict_proba(X_test_2)
model_final_predict_proba_np_default_t2 = model_final_predict_proba_np_t2[:,1]

In [66]:
Test2["Prediction_Prob"] = model_final_predict_proba_np_default_t2

<ipython-input-66-2c1e17fe8ead>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Test2["Prediction_Prob"] = model_final_predict_proba_np_default_t2
<ipython-input-66-2c1e17fe8ead>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test2["Prediction_Prob"] = model_final_predict_proba_np_default_t2


In [67]:
Test2[["target", "Prediction_Prob",]].head(10)

,target,Prediction_Prob
270,0,0.677444
271,0,0.677444
272,0,0.677444
273,0,0.677444
274,0,0.677444
275,0,0.746368
276,0,0.746368
277,0,0.764260
278,0,0.684759
279,0,0.684759


## Looking for possible best thresholds

In [68]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb

y_pred_proba = model_final.predict_proba(X_test_1)[:, 1]

# Step 4: Choose thresholds
thresholds = np.arange(0.1, 1, 0.1)

# Step 5: Calculate metrics
results = []
for threshold in thresholds:
    y_pred = (y_pred_proba > threshold).astype(int)
    accuracy = accuracy_score(Y_test_1, y_pred)
    precision = precision_score(Y_test_1, y_pred)
    recall = recall_score(Y_test_1, y_pred)
    f1 = f1_score(Y_test_1, y_pred)
    results.append((threshold, accuracy, precision, recall, f1))

# Step 6: Select best threshold
best_threshold_index = np.argmax([result[4] for result in results])
best_threshold = results[best_threshold_index][0]

# Print results in a table
print("| Threshold | Accuracy | Precision | Recall | F1 Score |")
print("|-----------|----------|-----------|--------|----------|")
for result in results:
    print(f"| {result[0]:.2f}       | {result[1]:.2f}     | {result[2]:.2f}      | {result[3]:.2f}   | {result[4]:.2f}     |")

print(f"\nBest Threshold: {best_threshold:.2f}")


| Threshold | Accuracy | Precision | Recall | F1 Score |
|-----------|----------|-----------|--------|----------|
| 0.10       | 0.06     | 0.06      | 1.00   | 0.11     |
| 0.20       | 0.06     | 0.06      | 1.00   | 0.11     |
| 0.30       | 0.06     | 0.06      | 1.00   | 0.11     |
| 0.40       | 0.06     | 0.06      | 1.00   | 0.11     |
| 0.50       | 0.06     | 0.06      | 1.00   | 0.11     |
| 0.60       | 0.06     | 0.06      | 1.00   | 0.11     |
| 0.70       | 0.54     | 0.05      | 0.40   | 0.10     |
| 0.80       | 0.94     | 0.00      | 0.00   | 0.00     |
| 0.90       | 0.94     | 0.00      | 0.00   | 0.00     |

Best Threshold: 0.10


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb

# Assuming you have already trained your XGBoost model and have validation/test data
# X_val: validation/test features, y_val: validation/test labels, model: trained XGBoost model

# Step 3: Predict probabilities
y_pred_proba = model_final.predict_proba(X_test_2)[:, 1]

# Step 4: Choose thresholds
thresholds = np.arange(0.1, 1, 0.1)

# Step 5: Calculate metrics
results = []
for threshold in thresholds:
    y_pred = (y_pred_proba > threshold).astype(int)
    accuracy = accuracy_score(Y_test_2, y_pred)
    precision = precision_score(Y_test_2, y_pred)
    recall = recall_score(Y_test_2, y_pred)
    f1 = f1_score(Y_test_2, y_pred)
    results.append((threshold, accuracy, precision, recall, f1))

# Step 6: Select best threshold
best_threshold_index = np.argmax([result[4] for result in results])
best_threshold = results[best_threshold_index][0]

# Print results in a table
print("| Threshold | Accuracy | Precision | Recall | F1 Score |")
print("|-----------|----------|-----------|--------|----------|")
for result in results:
    print(f"| {result[0]:.2f}       | {result[1]:.2f}     | {result[2]:.2f}      | {result[3]:.2f}   | {result[4]:.2f}     |")

print(f"\nBest Threshold: {best_threshold:.2f}")


| Threshold | Accuracy | Precision | Recall | F1 Score |
|-----------|----------|-----------|--------|----------|
| 0.10       | 0.21     | 0.21      | 1.00   | 0.35     |
| 0.20       | 0.21     | 0.21      | 1.00   | 0.35     |
| 0.30       | 0.21     | 0.21      | 1.00   | 0.35     |
| 0.40       | 0.21     | 0.21      | 1.00   | 0.35     |
| 0.50       | 0.21     | 0.21      | 1.00   | 0.35     |
| 0.60       | 0.21     | 0.21      | 1.00   | 0.35     |
| 0.70       | 0.53     | 0.27      | 0.75   | 0.40     |
| 0.80       | 0.79     | 0.00      | 0.00   | 0.00     |
| 0.90       | 0.79     | 0.00      | 0.00   | 0.00     |

Best Threshold: 0.70


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [70]:
Test2_copy1 = Test2.copy()
Test2_copy2 = Test2.copy()
Test2_copy3 = Test2.copy()
Test1_copy1 = Test1.copy()
Test1_copy2 = Test1.copy()
Test1_copy3 = Test1.copy()

In [71]:
Test1_copy1['Decision'] = np.where(Test1_copy1.Prediction_Prob > 0.75, 'buy', 'hold or sell')
Test1_copy1['true_or_false'] = np.where((Test1_copy1.Decision == 'buy') & (Test1_copy1.target == 1), 'true', np.where((Test1_copy1.Decision == 'hold or sell') & (Test1_copy1.target == 0), 'true', 'false'))
Test1_copy1[["target", "Prediction_Prob",'Decision','true_or_false',]].tail(10)

,target,Prediction_Prob,Decision,true_or_false
260,0,0.693435,hold or sell,true
261,0,0.693435,hold or sell,true
262,1,0.693435,hold or sell,false
263,1,0.688685,hold or sell,false
264,1,0.764260,buy,true
265,0,0.764260,buy,false
266,0,0.746368,hold or sell,true
267,0,0.764260,buy,false
268,0,0.684759,hold or sell,true
269,0,0.764260,buy,false


In [72]:
true_percent1_1 = Test1_copy1['true_or_false'].value_counts(normalize=True)['true'] * 100
false_percent1_1 = Test1_copy1['true_or_false'].value_counts(normalize=True)['false'] * 100

print("Percentage of true predictions:", true_percent1_1)
print("Percentage of false predictions:", false_percent1_1)

Percentage of true predictions: 60.24096385542169
Percentage of false predictions: 39.75903614457831


In [73]:
Test1_copy2['Decision'] = np.where(Test1_copy2.Prediction_Prob > 0.7, 'buy', 'hold or sell')
Test1_copy2['true_or_false'] = np.where((Test1_copy2.Decision == 'buy') & (Test1_copy2.target == 1), 'true', np.where((Test1_copy2.Decision == 'hold or sell') & (Test1_copy2.target == 0), 'true', 'false'))
Test1_copy2[["target", "Prediction_Prob",'Decision','true_or_false',]].tail(10)

,target,Prediction_Prob,Decision,true_or_false
260,0,0.693435,hold or sell,true
261,0,0.693435,hold or sell,true
262,1,0.693435,hold or sell,false
263,1,0.688685,hold or sell,false
264,1,0.764260,buy,true
265,0,0.764260,buy,false
266,0,0.746368,buy,false
267,0,0.764260,buy,false
268,0,0.684759,hold or sell,true
269,0,0.764260,buy,false


In [74]:
true_percent1_2 = Test1_copy2['true_or_false'].value_counts(normalize=True)['true'] * 100
false_percent1_2 = Test1_copy2['true_or_false'].value_counts(normalize=True)['false'] * 100

print("Percentage of true predictions:", true_percent1_2)
print("Percentage of false predictions:", false_percent1_2)

Percentage of true predictions: 54.21686746987952
Percentage of false predictions: 45.78313253012048


In [75]:
Test1_copy3['Decision'] = np.where(Test1_copy3.Prediction_Prob > 0.1, 'buy', 'hold or sell')
Test1_copy3['true_or_false'] = np.where((Test1_copy3.Decision == 'buy') & (Test1_copy3.target == 1), 'true', np.where((Test1_copy3.Decision == 'hold or sell') & (Test1_copy3.target == 0), 'true', 'false'))
Test1_copy3[["target", "Prediction_Prob",'Decision','true_or_false',]].tail(10)

,target,Prediction_Prob,Decision,true_or_false
260,0,0.693435,buy,false
261,0,0.693435,buy,false
262,1,0.693435,buy,true
263,1,0.688685,buy,true
264,1,0.764260,buy,true
265,0,0.764260,buy,false
266,0,0.746368,buy,false
267,0,0.764260,buy,false
268,0,0.684759,buy,false
269,0,0.764260,buy,false


In [76]:
true_percent1_3 = Test1_copy3['true_or_false'].value_counts(normalize=True)['true'] * 100
false_percent1_3 = Test1_copy3['true_or_false'].value_counts(normalize=True)['false'] * 100

print("Percentage of true predictions:", true_percent1_3)
print("Percentage of false predictions:", false_percent1_3)

Percentage of true predictions: 6.024096385542169
Percentage of false predictions: 93.97590361445783


In [77]:
Test2_copy1['Decision'] = np.where(Test2_copy1.Prediction_Prob > 0.75, 'buy', 'hold or sell')
Test2_copy1['true_or_false'] = np.where((Test2_copy1.Decision == 'buy') & (Test2_copy1.target == 1), 'true', np.where((Test2_copy1.Decision == 'hold or sell') & (Test2_copy1.target == 0), 'true', 'false'))
Test2_copy1[["target", "Prediction_Prob",'Decision','true_or_false',]].tail(10)


,target,Prediction_Prob,Decision,true_or_false
317,0,0.677444,hold or sell,true
318,0,0.677444,hold or sell,true
319,1,0.677444,hold or sell,false
320,0,0.764260,buy,false
321,0,0.677444,hold or sell,true
322,0,0.677444,hold or sell,true
323,0,0.677444,hold or sell,true
324,0,0.700818,hold or sell,true
325,0,0.700818,hold or sell,true
326,0,0.700818,hold or sell,true


In [78]:
true_percent2_1 = Test2_copy1['true_or_false'].value_counts(normalize=True)['true'] * 100
false_percent2_1 = Test2_copy1['true_or_false'].value_counts(normalize=True)['false'] * 100

print("Percentage of true predictions:", true_percent2_1)
print("Percentage of false predictions:", false_percent2_1)


Percentage of true predictions: 80.7017543859649
Percentage of false predictions: 19.298245614035086


In [79]:
Test2_copy2['Decision'] = np.where(Test2_copy2.Prediction_Prob > 0.7, 'buy', 'hold or sell')
Test2_copy2['true_or_false'] = np.where((Test2_copy2.Decision == 'buy') & (Test2_copy2.target == 1), 'true', np.where((Test2_copy2.Decision == 'hold or sell') & (Test2_copy2.target == 0), 'true', 'false'))
Test2_copy2[["target", "Prediction_Prob",'Decision','true_or_false',]].tail(10)

,target,Prediction_Prob,Decision,true_or_false
317,0,0.677444,hold or sell,true
318,0,0.677444,hold or sell,true
319,1,0.677444,hold or sell,false
320,0,0.764260,buy,false
321,0,0.677444,hold or sell,true
322,0,0.677444,hold or sell,true
323,0,0.677444,hold or sell,true
324,0,0.700818,buy,false
325,0,0.700818,buy,false
326,0,0.700818,buy,false


In [80]:
true_percent2_2 = Test2_copy2['true_or_false'].value_counts(normalize=True)['true'] * 100
false_percent2_2 = Test2_copy2['true_or_false'].value_counts(normalize=True)['false'] * 100

print("Percentage of true predictions:", true_percent2_2)
print("Percentage of false predictions:", false_percent2_2)


Percentage of true predictions: 52.63157894736842
Percentage of false predictions: 47.368421052631575


In [81]:
Test2_copy3['Decision'] = np.where(Test2_copy3.Prediction_Prob > 0.1, 'buy', 'hold or sell')
Test2_copy3['true_or_false'] = np.where((Test2_copy3.Decision == 'buy') & (Test2_copy3.target == 1), 'true', np.where((Test2_copy3.Decision == 'hold or sell') & (Test2_copy3.target == 0), 'true', 'false'))
Test2_copy3[["target", "Prediction_Prob",'Decision','true_or_false',]].tail(10)

,target,Prediction_Prob,Decision,true_or_false
317,0,0.677444,buy,false
318,0,0.677444,buy,false
319,1,0.677444,buy,true
320,0,0.764260,buy,false
321,0,0.677444,buy,false
322,0,0.677444,buy,false
323,0,0.677444,buy,false
324,0,0.700818,buy,false
325,0,0.700818,buy,false
326,0,0.700818,buy,false


In [82]:
true_percent2_3 = Test2_copy3['true_or_false'].value_counts(normalize=True)['true'] * 100
false_percent2_3 = Test2_copy3['true_or_false'].value_counts(normalize=True)['false'] * 100

print("Percentage of true predictions:", true_percent2_3)
print("Percentage of false predictions:", false_percent2_3)

Percentage of true predictions: 21.052631578947366
Percentage of false predictions: 78.94736842105263


## Table with results of accuracy of the thresholds

In [85]:
# Dataframe with all the percentages of "true" and "false" of Test1_copy1, Test1_copy2, Test1_copy3, Test2_copy1, Test2_copy2, and Test2_copy3, with its perspective true_percent1_1, false_percent1_1 and add the "%"

import pandas as pd
df = pd.DataFrame({
    'Test': ['Test1_copy1', 'Test1_copy2', 'Test1_copy3', 'Test2_copy1', 'Test2_copy2', 'Test2_copy3'],
    "Threshold" : ['.75', '.60', '.10', '.75', '.60', '.10'],
    'true_percent': [true_percent1_1, true_percent1_2, true_percent1_3, true_percent2_1, true_percent2_2, true_percent2_3],
    'false_percent': [false_percent1_1, false_percent1_2, false_percent1_3, false_percent2_1, false_percent2_2, false_percent2_3]
})

df['true_percent'] = df['true_percent'].apply(lambda x: f'{x:.2f}%')
df['false_percent'] = df['false_percent'].apply(lambda x: f'{x:.2f}%')

df


,Test,Threshold,true_percent,false_percent
0,Test1_copy1,.75,60.24%,39.76%
1,Test1_copy2,.60,54.22%,45.78%
2,Test1_copy3,.10,6.02%,93.98%
3,Test2_copy1,.75,80.70%,19.30%
4,Test2_copy2,.60,52.63%,47.37%
5,Test2_copy3,.10,21.05%,78.95%
